# ZChop

*Replace small numbers with zero*

`zchop(x)` replaces numbers in `x` that are close to zero with zero.

```zchop(x)``` returns 0 if abs(x) is smaller than 1e-14, and x otherwise.

```zchop(x,eps)``` uses eps rather than 1e-14

```zchop!(a,eps)``` works inplace on Array a.


### Examples


### Example 1: Inverse Fourier transform

(If FFTW is not installed, you can install it with `Pkg.add("FFTW")`)

Performing the discrete Fourier transform of a real sequence, followed by the inverse transform returns a complex array. Due to limited numerical accuracy, some numbers that should be zero are instead very close to zero.

In [1]:
using FFTW
real_sequence = [2,1,1,0,0,0,0]
result = real_sequence |> fft |> ifft

7-element Array{Complex{Float64},1}:
                    2.0 + 0.0im
                    1.0 + 0.0im
                    1.0 + 0.0im
  1.527827807198305e-17 + 0.0im
  5.727136726909545e-18 + 0.0im
                    0.0 + 0.0im
 -6.344131569286608e-17 + 0.0im

Load the `ZChop` package, and replace these small numbers with `0`

In [2]:
using ZChop
zchop(result)

7-element Array{Complex{Float64},1}:
 2.0 + 0.0im
 1.0 + 0.0im
 1.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im

Trying to directly convert the result to an array of integers fails.

In [3]:
Int[result...]

LoadError: InexactError: Int64(Int64, 1.527827807198305e-17)

But, we can do it with `zchop`.

In [4]:
Int[zchop(result)...]

7-element Array{Int64,1}:
 2
 1
 1
 0
 0
 0
 0

### Example 2

In [5]:
result = exp.(range(1, stop=4, length=4) * pi * im)

4-element Array{Complex{Float64},1}:
 -1.0 + 1.2246467991473532e-16im
  1.0 - 2.4492935982947064e-16im
 -1.0 + 3.6739403974420594e-16im
  1.0 - 4.898587196589413e-16im 

In [6]:
Int[zchop(result)...]

4-element Array{Int64,1}:
 -1
  1
 -1
  1

### Example 3

In [7]:
using SparseArrays
a = sparse([ [1.0,1e-16]  [1e-16, 1.0]])

2×2 SparseMatrixCSC{Float64,Int64} with 4 stored entries:
  [1, 1]  =  1.0
  [2, 1]  =  1.0e-16
  [1, 2]  =  1.0e-16
  [2, 2]  =  1.0

In [8]:
dropzeros(zchop(a))

2×2 SparseMatrixCSC{Float64,Int64} with 2 stored entries:
  [1, 1]  =  1.0
  [2, 2]  =  1.0

This is not efficient. It would be better to use `dropzeros` directly. But, it seems that `dropzeros` does not support specifying a tolerance.

### More details

The type of the numbers is preserved.  For instance, complex numbers
with imaginary part near zero are not replaced with real numbers.

`zchop` works on complex and rational numbers, arrays, and some other structures.
`zchop` descends into structures, chopping numbers, and acting as the
the identity on anything that can't be sensibly compared to `eps`.

In [9]:
a = Any[[1e-15, "dog", (BigFloat(10.0))^-15, complex(1e-15,1), 1 // 10^15],
         [[2,3] [4,1e-15]] ];

zchop(a)

2-element Array{Any,1}:
 Any[0.0, "dog", 0.0, 0.0+1.0im, 0//1]
 [2.0 4.0; 3.0 0.0]                   

#### Version and Date

In [10]:
VERSION

v"0.7.0-rc1.15"

In [11]:
using Dates
Dates.now()

2018-08-03T11:03:32.339